In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark import broadcast, SparkConf

In [2]:
import os
# You need to update these to your real paths!
dataRoot = os.getenv("DATA_ROOT", 'file:///run/determined/workdir/data')

In [3]:
import random
import time

if "sc" in globals():
    sc.stop()
    
conf = SparkConf().setAppName("Retail Analytics")

In [4]:
### Configure the parameters based on your dataproc cluster ###
conf.set("spark.scheduler.minRegisteredResourcesRatio", "0.0")

conf.set("spark.executor.instances", "8")
conf.set("spark.executor.cores", "4")
conf.set("spark.executor.memory", "8192m")
conf.set("spark.executor.memoryOverhead", "4915m")
conf.set("spark.yarn.executor.launch.excludeOnFailure.enabled",True)

conf.set("spark.driver.maxResultSize", "8g")
conf.set("spark.driver.memory", "10g")

conf.set("spark.sql.files.maxPartitionBytes", "512m")
conf.set("spark.sql.broadcastTimeout", "700")
conf.set("spark.sql.shuffle.partitions", "500")
conf.set("spark.sql.adaptive.enabled", "true")
conf.set("spark.sql.autoBroadcastJoinThreshold", "300M")
conf.set("spark.sql.adaptive.advisoryPartitionSizeInBytes", "128M")
conf.set("spark.sql.adaptive.coalescePartitions.minPartitionNum", "1")
conf.set("spark.dynamicAllocation.enabled", "false")


In [5]:
### Enable RAPIDS config

conf.set("spark.plugins", "com.nvidia.spark.SQLPlugin")
conf.set("spark.executor.resource.gpu.amount", "1")
# conf.set("spark.task.resource.gpu.amount", "1") # This is the part that's failing...

conf.set("spark.plugins", "com.nvidia.spark.SQLPlugin")
conf.set("spark.rapids.sql.concurrentGpuTasks", "2")
conf.set("spark.rapids.memory.pinnedPool.size", "4096m")
conf.set("spark.rapids.memory.host.spillStorageSize", "4g")
conf.set("spark.rapids.sql.multiThreadedRead.numThreads", "40")
conf.set("spark.rapids.sql.castDecimalToString.enabled",True)
conf.set("spark.rapids.sql.castStringToTimestamp.enabled",True)
conf.set("spark.rapids.sql.expression.PercentRank",False)
conf.set("spark.rapids.sql.castDecimalToString.enabled",True)
conf.set("spark.rapids.sql.format.json.read.enabled",True)
conf.set("spark.rapids.sql.explain",None)
conf.set("spark.rapids.sql.enabled",True)

In [6]:
spark = SparkSession.builder \
                    .config(conf=conf) \
                    .getOrCreate()

In [7]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window

start = time.time()

def clean_data(df):
    # remove missing values
    df = df.dropna()
    # remove duplicate data
    df = df.dropDuplicates()
    return df


def read_data(spark, format, file_path):
    if format=="csv":
        return spark.read.format(format).load(file_path,header=True)
    else:
        return spark.read.format(format).load(file_path)

# read sales data
sales_df = read_data(spark, "csv", dataRoot+"/raw/sales/")

# read stock data
stock_df = read_data(spark, "json", dataRoot+"/raw/stock/")

# read supplier data
supplier_df = read_data(spark, "json", dataRoot+"/raw/supplier/")

# read customer data
customer_df = read_data(spark, "csv", dataRoot+"/raw/customer/")

# read market data
market_df = read_data(spark, "csv", dataRoot+"/raw/market/")

# read logistic data
logistic_df = read_data(spark, "csv", dataRoot+"/raw/logistic/")


# data cleaning
sales_df = clean_data(sales_df)
stock_df = clean_data(stock_df)
supplier_df = clean_data(supplier_df)
customer_df = clean_data(customer_df)
market_df = clean_data(market_df)
logistic_df = clean_data(logistic_df)


# convert date columns to date type
sales_df = sales_df.withColumn("date_of_sale", to_date(col("date_of_sale")))
stock_df = stock_df.withColumn("date_received", to_date(col("date_received")))
supplier_df = supplier_df.withColumn("date_ordered", to_date(col("date_ordered")))

# standardize case of string columns
sales_df = sales_df.withColumn("product_name", upper(col("product_name")))
stock_df = stock_df.withColumn("product_name", upper(col("product_name")))
stock_df = stock_df.withColumn("location", upper(col("location")))
supplier_df = supplier_df.withColumn("product_name", upper(col("product_name")))
customer_df = customer_df.withColumn("customer_name", upper(col("customer_name")))
market_df = market_df.withColumn("product_name", upper(col("product_name")))
logistic_df = logistic_df.withColumn("product_name", upper(col("product_name")))

# remove leading and trailing whitespaces
sales_df = sales_df.withColumn("product_name", trim(col("product_name")))
stock_df = stock_df.withColumn("location", trim(col("location")))

supplier_df = supplier_df.withColumn("product_name", trim(col("product_name")))
customer_df = customer_df.withColumn("customer_name", trim(col("customer_name")))
market_df = market_df.withColumn("product_name", trim(col("product_name")))
logistic_df = logistic_df.withColumn("product_name", trim(col("product_name")))

# check for invalid values
sales_df = sales_df.filter(col("product_name").isNotNull())
stock_df = stock_df.filter(col("location").isNotNull())
customer_df = customer_df.filter(col("gender").isin("male","female"))
market_df = market_df.filter(col("product_name").isNotNull())
logistic_df = logistic_df.filter(col("product_name").isNotNull())

#drop extra columns
market_df = market_df.drop("price")
supplier_df = supplier_df.drop("price")

# join all data
data_int = sales_df.join(stock_df, "product_name","leftouter").join(supplier_df, "product_name","leftouter").join(market_df, "product_name","leftouter").join(logistic_df, "product_name","leftouter").join(customer_df, "customer_id","leftouter")  

# write the cleaned data
data_int.write.format("parquet").mode("overwrite").save(dataRoot+"/cleaned/")

end = time.time()

print("Time taken on GPU for Data Cleaning: ", end - start)



Time taken on GPU for Data Cleaning:  33.29042196273804


In [8]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window

#DO VARIOUS RETAIL DATA ANALYTICS 

start = time.time()

# read cleaned data

data = spark.read.format("parquet").load(dataRoot+"/cleaned/")

#Case when statement to create a new column to indicate whether the product is perishable or not:

data = data.withColumn("perishable", when(col("shelf_life") <= 30, "yes").otherwise("no"))

# You can use the when() and otherwise() functions to create new columns based on certain conditions:

data = data.withColumn("sales_status", when(col("quantity_sold") > 50, "good").otherwise("bad"))

# create a window to perform time series analysis
window = Window.partitionBy("product_name").orderBy("date_of_sale")

# calculate the rolling average of sales for each product
time_series_df = data.withColumn("rolling_avg_sales", avg("quantity_sold").over(window))

# use window function for forecasting

forecast_df = time_series_df.withColumn("prev_sales", lag("rolling_avg_sales").over(window))\
    .withColumn("next_sales", lead("rolling_avg_sales").over(window))


# Calculate the average price of a product, grouped by supplier
forecast_df.groupBy("sup_id").agg({"price": "avg"}).show()


# Calculate the total quantity in stock and total sales by supplier
forecast_df.groupBy("sup_id").agg({"quantity_in_stock": "sum", "price": "sum"}).show()

#Calculate the number of perishable v/s non-perishable product per location
forecast_df.groupBy("perishable").agg({"perishable": "count"}).show()


#Calculate number of good v/s bad sales status per location
forecast_df.groupBy("sales_status").agg({"sales_status": "count"}).show()

# Count the number of sales that contain a 10% off promotion
countt = forecast_df.filter(forecast_df["contains_promotion"].contains("10% off")).count()
print(countt)
# Perform some complex analysis on the DataFrame

# Calculate the total sales, quantity sold by product and location
total_sales_by_product_location = forecast_df.groupBy("product_name", "location").agg(sum("price").alias("total_price"),sum("quantity_ordered").alias("total_quantity_sold"),avg("quantity_sold").alias("avg_quantity_sold")).sort(desc("total_price"))

# Group the data by product_name
grouped_df = forecast_df.groupBy("product_name")

#Sum the quantity_in_stock, quantity_ordered, quantity_sold, and (price * quantity_sold) for each group
aggregated_df = grouped_df.agg(sum("quantity_in_stock").alias("total_quantity_in_stock"),avg("price").alias("average_price"),sum("quantity_ordered").alias("total_quantity_ordered"),sum("quantity_sold").alias("total_quantity_sold"),sum(col("price") * col("quantity_sold")).alias("total_sales"),sum("prev_sales").alias("total_prev_sales"),sum("next_sales").alias("total_next_sales"),).sort(desc("total_sales"))

#WRITE THE AGGREGATES TO DISK
aggregated_df.write.format("parquet").mode("overwrite").save(dataRoot+"/app/data.parquet")
total_sales_by_product_location.write.format("parquet").mode("overwrite").save(dataRoot+"/app1/data.parquet")

end = time.time()

print("Time taken on GPU for Data Analysis: ", end - start)

+--------+------------------+
|  sup_id|        avg(price)|
+--------+------------------+
|s_489730| 49.52406329057808|
|s_406709| 69.81079196518795|
|s_115079| 85.94399912664427|
|s_344666| 97.59009379382898|
|s_285915|14.294712207062046|
| s_76565| 5.437999719320045|
|s_140303| 87.53240260474355|
|s_476437|60.546143121865114|
|s_119978| 69.73744545566842|
|s_236217| 46.73458879110069|
|s_320400| 82.35581718551407|
|  s_4063| 99.77199821260102|
|s_157361| 49.94368539159035|
| s_65960|  44.7196913589874|
|s_465221| 39.55720484422409|
|s_268747| 41.03514491012048|
| s_20835|34.089356272623064|
|s_495773| 85.89353051404636|
|s_400631| 1.130315066427185|
|s_226779| 46.68096597421045|
+--------+------------------+
only showing top 20 rows

+--------+------------------+----------------------+
|  sup_id|        sum(price)|sum(quantity_in_stock)|
+--------+------------------+----------------------+
|s_489730| 49.52406329057808|                   462|
|s_406709| 69.81079196518795|             

In [9]:
spark.stop()